## Process Scrapy Result

In [30]:
import json

file_name = "./linkedin_url.jl"
with open(file_name, "r") as f:
    lines = f.readlines()
    data = [json.loads(line) for line in lines]
data

[{'name': None,
  'link': 'https://www.linkedin.com/jobs/view/software-engineer-i-nst-at-nintendo-1813083152?refId=0229259c-ffff-4420-9193-74bcae023058&position=1&pageNum=0&trk=public_jobs_job-result-card_result-card_full-click'},
 {'name': 'Nintendo',
  'link': 'https://www.linkedin.com/company/nintendo?trk=public_jobs_job-result-card_result-card_subtitle-click'},
 {'name': None,
  'link': 'https://www.linkedin.com/jobs/view/software-engineer-at-venmo-1806370829?refId=0229259c-ffff-4420-9193-74bcae023058&position=2&pageNum=0&trk=public_jobs_job-result-card_result-card_full-click'},
 {'name': 'Venmo',
  'link': 'https://www.linkedin.com/company/venmo?trk=public_jobs_job-result-card_result-card_subtitle-click'},
 {'name': None,
  'link': 'https://www.linkedin.com/jobs/view/software-engineer-full-stack-at-knock-1817639742?refId=0229259c-ffff-4420-9193-74bcae023058&position=3&pageNum=0&trk=public_jobs_job-result-card_result-card_full-click'},
 {'name': 'Knock',
  'link': 'https://www.link

In [38]:
companies = {}
num_company, num_job = 0, 0
for idx in range(len(data)//2):
    job, company = data[idx * 2], data[idx * 2 + 1]
#     try:
    assert job["name"] is None
    assert company["name"] is not None

    company_url = company["link"].split("?")[0]
    company_name = company["name"]
    company_id = company_url.split("/")[-1]

    job_idx = job["link"].split("?")[0].split("-")[-1]
    job_url = f"https://www.linkedin.com/jobs/view/{job_idx}"

    if company_id in companies:
        # this company has appeared before
        if job_url not in companies[company_id]["jobs"]:
            companies[company_id]["jobs"].append(job_url)
            num_job += 1
    else:
        companies[company_id] = {
            "name": company_name,
            "company_url": company_url,
            "jobs": [job_url]
        }
        num_company += 1
        num_job += 1
#     except:
#         continue
companies

{'nintendo': {'name': 'Nintendo',
  'company_url': 'https://www.linkedin.com/company/nintendo',
  'jobs': ['https://www.linkedin.com/jobs/view/1813083152']},
 'venmo': {'name': 'Venmo',
  'company_url': 'https://www.linkedin.com/company/venmo',
  'jobs': ['https://www.linkedin.com/jobs/view/1806370829']},
 'knock-inc': {'name': 'Knock',
  'company_url': 'https://www.linkedin.com/company/knock-inc',
  'jobs': ['https://www.linkedin.com/jobs/view/1817639742']},
 'identityiq': {'name': 'IDIQ',
  'company_url': 'https://www.linkedin.com/company/identityiq',
  'jobs': ['https://www.linkedin.com/jobs/view/1812584277']},
 'mastercard': {'name': 'Mastercard',
  'company_url': 'https://www.linkedin.com/company/mastercard',
  'jobs': ['https://www.linkedin.com/jobs/view/1732205659']},
 'papaya-studio': {'name': 'Papaya Studio',
  'company_url': 'https://www.linkedin.com/company/papaya-studio',
  'jobs': ['https://www.linkedin.com/jobs/view/1817972514']},
 'godaddy': {'name': 'GoDaddy',
  'compan

In [39]:
# Note: this is parsed from 500 entries: 60 companies and ~120 
num_company, num_job

(49, 53)

## Company Extractor

In [40]:
list(companies.keys()), len(list(companies.keys()))

(['nintendo',
  'venmo',
  'knock-inc',
  'identityiq',
  'mastercard',
  'papaya-studio',
  'godaddy',
  'paypal',
  'blend',
  'haku',
  'allscripts',
  'time',
  'hclsoftware',
  'github',
  'seres-ev',
  'american-express',
  'tesla-motors',
  'tiny-spec-inc',
  'optum',
  'volkswagen-of-america-inc',
  'red-meters',
  'extrahop-networks',
  'grammarly',
  'parrylabs',
  'dcca',
  'sony-electronics',
  'slab',
  'telspan-data',
  'fidelity-investments',
  'idemiagroup',
  'wits-solutions',
  'avacend-inc',
  'clearedjobs-net',
  'chewy-com',
  'ecstechhq',
  'logan-britton',
  'boeing',
  'as&t-srl',
  'amazon',
  'frank-recruitment-group',
  'truistfinancialcorporation',
  'expedent-corp.',
  'intellicomm-inc-',
  'amkor-technology',
  'apex-systems',
  'getcruise',
  'arena-technical-resources',
  'jobs-interviewing-now-from-hcs',
  'green-key-resources'],
 49)

In [41]:
import os
os.environ['LI_AT'] = "AQEDATAWR74FZprSAAABcVyurrEAAAFxgLsysVYAPk-jSm_eJE5uDg0oFdJpY_7_OqTevgRN0ZxfrHgg3MtwaF--Sz1FMO61LoDScJ_s8TEGUvnkv3JSYeRSqLMpKG9s17m7GGqDQ7RDz-MZD7kclChR"

In [44]:
# POC of one company info
# driver.get("https://www.linkedin.com/company/fortress-power/about")

from scrape_linkedin import CompanyScraper
import pandas as pd
import time

# LIST YOUR COMPANIES HERE
my_company_list = list(companies.keys())

company_data = []

with CompanyScraper() as scraper:
    # Get each company's overview, add to company_data list
    for name in my_company_list:
        try:
            overview = scraper.scrape(company=name).overview
            overview['company_name'] = name
            company_data.append(overview)
            time.sleep(5.0)
        except:
            continue

# Turn into dataframe for easy csv output
df = pd.DataFrame(company_data)
df.to_csv('out.csv', index=False)

[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Company size
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>]
[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>]
{'description': 'At Nintendo we are proud to be working for the leading company in our industry. We are strongly committed to producing and marketing t

[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Company size
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Founded
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Specialties
    </dt>]
[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-

[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Company size
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Founded
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Specialties
    </dt>]
[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-

[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Company size
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Founded
    </dt>]
[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t

[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Company size
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>]
[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>]
{'description': 'Optum is a health services and innovation company on a mission to help people live healthier lives and to help make the health system work better for everyone.\n\nOptum, part of the UnitedHealth Group family of businesses, is powering modern health care by connecting and serving the whole health system across 150 countries. We combine

...

[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Company size
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Specialties
    </dt>]
[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-bla

[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Phone
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Company size
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Founded
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Specialties
    </dt>]
[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Phone
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">

[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Company size
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Founded
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Specialties
    </dt>]
[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-

[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Company size
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Founded
    </dt>]
[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Founded
    </dt>]
{'description': 'Expedent Corp is a private equity consortium Firm providing IT Staffing Services to Fortune 500 clients across the US , we believe that it’s an exceptio

[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Company size
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Type
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Founded
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Specialties
    </dt>]
[<dt class="org-page-details__definition-term t-14 t-black t-bold">
      Website
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
      Industry
    </dt>, <dt class="org-page-details__definition-term t-14 t-black t-bold">
        Headquarters
      </dt>, <dt class="org-page-details__definition-term t-14 t-

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome('/usr/local/bin/chromedriver')
driver.get("https://www.linkedin.com")
# Log-in required and re-use credential
# manual

# End of session: manual close
driver.close()

In [6]:
# Use Selenium to scrape company given url
def scrape_company_detail(company_url):
    pass

In [7]:
def scrape_job_detail(company_url):
    pass

In [8]:
# Use Selenium to scrape job details given url